In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pprint import pprint

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [3]:
dataset = 'survey_report_concat_common_questions.csv'
df = pd.read_csv(dataset, dtype=object)

In [4]:
df.head()

,year,original_missing_values_per_row_percentage,country,programming_languages,programming_languages_other,years_experience,occupation,age,os,job_satisfaction,company_size,want_work_language,gender,education,database_desire_work,database_worked_with,undergrad_major,remote_status,industry
0,2011,18.18,Africa,JavaScript,NaN,<2,Web Application Developer,< 20,Linux,FML,Start Up (1-25),NaN,NaN,NaN,NaN,NaN,NaN,NaN,Consulting
1,2011,27.27,Other Europe,Java;SQL;C,NaN,41310,Server Programmer,25-29,Windows 7,So happy it hurts,Mature Small Business (25-100),NaN,NaN,NaN,NaN,NaN,NaN,NaN,Software Products
2,2011,45.45,India,Java;JavaScript;SQL,NaN,41435,Server Programmer,25-29,Linux,NaN,Mid Sized (100-999),NaN,NaN,NaN,NaN,NaN,NaN,NaN,Software Products
3,2011,9.09,Germany,Java,Haskell,41310,Student,< 20,Linux,I enjoy going to work,Student,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Foundation / Non-Profit
4,2011,13.64,Other Asia,Java;JavaScript;CSS;PHP;SQL;C++;C;Perl,NaN,11,"Executive (VP of Eng, CTO, CIO, etc.)",35-39,Linux,It pays the bills,Start Up (1-25),NaN,NaN,NaN,NaN,NaN,NaN,NaN,Software Products


In [5]:
#df.programming_languages_other = df.programming_languages_other.str.lower().str.replace(',', ';').str.replace('.js', 'js').str.split(';')
df.programming_languages = df.programming_languages.str.split(';')

In [6]:
df_language_explode = df.explode('programming_languages')

In [7]:
df_language_explode['language'] = df_language_explode.programming_languages.str.strip()

In [8]:
df_language_explode.head()

,year,original_missing_values_per_row_percentage,country,programming_languages,programming_languages_other,years_experience,occupation,age,os,job_satisfaction,company_size,want_work_language,gender,education,database_desire_work,database_worked_with,undergrad_major,remote_status,industry,language
0,2011,18.18,Africa,JavaScript,NaN,<2,Web Application Developer,< 20,Linux,FML,Start Up (1-25),NaN,NaN,NaN,NaN,NaN,NaN,NaN,Consulting,JavaScript
1,2011,27.27,Other Europe,Java,NaN,41310,Server Programmer,25-29,Windows 7,So happy it hurts,Mature Small Business (25-100),NaN,NaN,NaN,NaN,NaN,NaN,NaN,Software Products,Java
1,2011,27.27,Other Europe,SQL,NaN,41310,Server Programmer,25-29,Windows 7,So happy it hurts,Mature Small Business (25-100),NaN,NaN,NaN,NaN,NaN,NaN,NaN,Software Products,SQL
1,2011,27.27,Other Europe,C,NaN,41310,Server Programmer,25-29,Windows 7,So happy it hurts,Mature Small Business (25-100),NaN,NaN,NaN,NaN,NaN,NaN,NaN,Software Products,C
2,2011,45.45,India,Java,NaN,41435,Server Programmer,25-29,Linux,NaN,Mid Sized (100-999),NaN,NaN,NaN,NaN,NaN,NaN,NaN,Software Products,Java


In [9]:
#pprint(df_language_explode.programming_languages.unique().tolist())
df_language_explode.programming_languages.value_counts().to_frame()

,language
JavaScript,228013
SQL,186763
Java,143905
Python,126451
C#,116025
PHP,98454
HTML/CSS,91647
C++,81756
C,71645
CSS,55159
